#### model I/O
> LangChain은 주요 모듈을 위한 확장 가능한 표준 인터페이스 및 외부 통합 기능을 제공
> - model I/O: 언어 모델과의 인터페이스

> - Retrieval: 애플리케이션별 데이터를 사용한 인터페이스
> - - Retrieval: 외부 데이터로 접근해 이를 모델에 어떻게 제공하는 것에 관한 것

> - chain: 일반적인 빌딩 블록 구성

> - memory: 체인 실행 간 지속적인 응용 프로그램 상태

> - callback: 모든 체인의 중간 단계 기록 및 스트리밍

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

# t = PromptTemplate.from_template("{country}의 수도는 어디야?")
t = PromptTemplate(
    template = "{country}의 수도는 어디야?",
    input_variables = ["country"],
)

t.format(country = "서울")

In [ ]:
# FewShotPromptTemplate
# Fewshot은 모델에게 예제들을 준다는 뜻과 같다. -> 더 나은 대답을 할 수 있도록 예제를 준다.

# FewShotPromptTemplate
# PromptTemplate

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_template = """
    Human: {question}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt.format(country= "korea")
chain = prompt | chat
chain.invoke({
    "country": "한국"
})

In [ ]:
# FewShotChatMessagePromptTemplate
# ChatPromptTemplate

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat_model = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{country}에 대해 설명해줘"),
    ("ai", "{answer}")
])

prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 지리 전문가야, 짧게 말해줘"),
    prompt,
    ("human", "{country}에 대해 설명해줘")
])

chain = final_prompt | chat
chain.invoke({"country": "Korea"})

In [ ]:
# LengthBasedExampleSelector

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector        

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
        
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

# example_selector = LengthBasedExampleSelector(
example_selector = RandomExampleSelector(
    examples=examples
    # example_prompt=example_prompt,
    # max_length=180
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country= "korea")

chain = prompt | chat
chain.invoke({"country": "Korea"})

In [ ]:
# Serialization and Composition

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt1 = load_prompt("./prompt.json")
prompt2 = load_prompt("./prompt.yaml")

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

a = prompt1.format(country="korea")
b = prompt2.format(country="korea")
print(a, "\n", b)

In [ ]:
# PipelinePromptTemplate: prompt가 많을 때 유용

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat_model = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


intro = PromptTemplate.from_template(
    """
    당신은 역할놀이 보조자입니다. 그리고 당신은 {character} 역을 맡아야 합니다.
"""
)

example = PromptTemplate.from_template(
    """
    이건 어떻게 말해야 하는지 예시야:
    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
        """
    시작!
    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
        """
    {intro}
    
    {example}
    
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]
full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

# 확인
# full_prompt.format(
#     character = "해적",
#     example_question = "어디에 있어?",
#     example_answer = "크하하하! 그건 비밀이다!",
#     question = "너가 가장 좋아하는 음식은 뭐야?"
# )

chain = full_prompt | chat_model
chain.invoke({
    "character": "해적",
    "example_question": "어디에 있어?",
    "example_answer": "크하하하! 그건 비밀이다!",
    "question": "너가 가장 좋아하는 음식은 뭐야?"
})

In [ ]:
# caching: 같은 질문을 계속 받으면 답변을 재사용해 비용을 절약할 수 있게 해준다.

from langchain.chat_models import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(True)
# 무슨 일을 하고 있는지에 대한 로그를 보여준다.

chat_model = ChatOpenAI(temperature=0.1)

chat_model.predict("토마토 스파게티는 어떻게 만들어?")

In [ ]:
chat_model.predict("토마토 스파게티는 어떻게 만들어?")

In [ ]:
# caching: 같은 질문을 계속 받으면 답변을 재사용해 비용을 절약할 수 있게 해준다.
from langchain.chat_models import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))

set_debug(True)

chat_model = ChatOpenAI(temperature=0.1)

chat_model.predict("토마토 스파게티는 어떻게 만들어?")

In [2]:
# Serialization
# 모델을 어떻게 저장하고 불러오는지

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(temperature=0.1)

with get_openai_callback() as usage:
    a = chat.predict("소주는 어떻게 만들어")
    b = chat.predict("김쫀떡 알아?")
    print(a, "\n", b, "\n")
    print(usage)

지는지 궁금하시군요. 소주는 주로 쌀, 보리, 감자 등의 원료를 발효시켜 만들어집니다. 먼저 선택한 곡물을 발효시켜 알코올을 얻은 후, 증류 과정을 거쳐 순수한 주정을 얻습니다. 이후 숙성을 거쳐 부드럽고 깔끔한 맛을 내는 과정을 거칩니다. 소주의 맛과 향은 발효 및 숙성 과정에서 나오는 효모와 박테리아의 영향을 받기 때문에 다양한 종류의 소주가 만들어집니다. 각 지역마다 특색 있는 소주가 만들어지는데, 한국의 전통주인 소주는 다양한 발효 및 숙성 기술을 통해 만들어지며 맛과 향이 다양하게 변화합니다. 
 네, 김쫀떡은 한국의 전통적인 떡으로, 쫄깃한 떡에 김가루를 묻혀 만든 떡입니다. 해물이나 쇠고기 등 다양한 재료를 넣어 만들기도 하며, 소금이나 간장 등으로 맛을 낼 수 있습니다. 맛있는 한국 전통 간식 중 하나입니다. 

Tokens Used: 458
	Prompt Tokens: 38
	Completion Tokens: 420
Successful Requests: 2
Total Cost (USD): $0.000897


In [5]:
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

# chat = OpenAI(temperature=0.1, max_tokens=450, model="gpt-3.5-turbo-16k")
# chat.save("model.json")

chat = load_llm("model.json")
chat 

/Users/jangtaehun/Desktop/study/nomad/Fullstack-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/jangtaehun/Desktop/study/nomad/Fullstack-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})